In [1]:
import os
import dask.dataframe as ddf

from dask.distributed import LocalCluster, Client

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
file_lst = [file.path for file in os.scandir("../dossier/") if file.is_file() and file.path.endswith(".csv")]

In [4]:
def read_files(files_list : list, **kwargs) -> ddf:
    """Read multiple files into a Dask DataFrame

    Args:
        files_list (list): List of files to read

    Returns:
        ddf: Delayed Dask DataFrame object
    """
    return ddf.read_csv(files_list, **kwargs)